In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold, LeaveOneOut, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Lasso, Ridge
from time import process_time
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv("DailyActivitiesPreProcessed.csv")

df2 = df[['Gender', 'Age', 'Occupation', 'Marital status', 'HouseHoldIncome', 'IncomeGroup',
          'Sleeping_duration_weekend', 'University_Class_duration_weekend', 'Study_duration_weekend',
          'Exercise_duration_weekend', 'Work_duration_weekend', 'Housework_duration_weekend',
          'Entertainment_duration_weekend', 'Personal_Care_duration_weekend', 'Eating_duration_weekend',
          'Socializing_duration_weekend', 'Religion_or_Spiritual_Activities_duration_weekend',
          'Shopping_duration_weekend', 'Cooking_duration_weekend', 'Family_Time_duration_weekend',
          'Sports_duration_weekend', 'Reading_or_Writing_duration_weekend', 'Nightlife_duration_weekend',
          'Travelling_duration_weekend', 'Volunteering_duration_weekend', 'Gardening_duration_weekend',
          'Entertainment_merged_duration_weekend', 'Personal_care_merged_duration_weekend',
          'Housework_merged_duration_weekend', 'Active_duration_weekend', 'Leisure_duration_weekend',
          'Neutral_duration_weekend', 'Productivity_score_weekend']]


# Assuming your DataFrame is named df

print(df2.columns.tolist())

# Columns to drop
columns_to_drop = [
    # Demographic information
    #'Gender', 'Age', 'Occupation', 'Marital status', 'HouseHoldIncome', 'IncomeGroup',

    # Weekday-specific activity columns (all 30-minute intervals)
    *[col for col in df.columns if col.startswith('Weekday_')],
    *[col for col in df.columns if 'duration_weekday' in col],

    # Fashion preferences
    'FashionPref', 'SmartDresserPref', 'StylePref',

    # Productivity score for weekday (target column, only drop if it's a feature, not the target)
    # Uncomment the next line if needed:
    'Productivity_score_weekday'
]

# Drop the columns
df2 = df2.drop(columns=columns_to_drop, errors='ignore')


print(df2.columns.tolist())

# Encode categorical variables
le = LabelEncoder()
df2['IncomeGroup'] = le.fit_transform(df2['IncomeGroup'])
df2 = pd.get_dummies(df2, columns=['Gender', 'Occupation', 'Marital status'], drop_first=True)

# Categorize the productivity score into low, moderate, and high
quartiles = df2['Productivity_score_weekend'].quantile([0.25, 0.5, 0.75])
def categorize_productivity(score):
    if score <= quartiles[0.25]:
        return 'low'
    elif score <= quartiles[0.75]:
        return 'moderate'
    else:
        return 'high'

df2['Productivity_group'] = df2['Productivity_score_weekend'].apply(categorize_productivity)
df2['Productivity_group'] = le.fit_transform(df2['Productivity_group'])


# Handle missing values
df2.dropna(inplace=True)

# # Drop columns to prevent leaking
# X = df2.drop(columns=['Productivity_score_weekday', 'Productivity_group', 'University_Class_duration_weekday', 'Study_duration_weekday', 'Exercise_duration_weekday', 'Work_duration_weekday',
# 'Housework_duration_weekday','Cooking_duration_weekday', 'Housework_merged_duration_weekday'])
# y = df2['Productivity_group']

# Prepare features and target
X = df2.drop(columns=['Productivity_group'])
y = df2['Productivity_group']

def evaluate_model(clf, X_train, X_test, y_train, y_test, model_type="classification"):
    start_time = process_time()
    clf.fit(X_train, y_train)
    end_time = process_time()
    total_training_time = end_time - start_time
    
    if model_type == "regression":
        predictions = np.round(clf.predict(X_test))
        train_predictions = np.round(clf.predict(X_train))
    else:
        predictions = clf.predict(X_test)
        train_predictions = clf.predict(X_train)
    
    print(f"Model: {clf.__class__.__name__}")
    print(f"Training time: {total_training_time:.2f} seconds")
    print(f"Train Accuracy: {accuracy_score(y_train, train_predictions)}")
    print(f"Test Accuracy: {accuracy_score(y_test, predictions)}")
    print(f"Precision: {precision_score(y_test, predictions, average='weighted')}")
    print(f"Recall: {recall_score(y_test, predictions, average='weighted')}")
    print(f"F1 Score: {f1_score(y_test, predictions, average='weighted')}")
    print(confusion_matrix(y_test, predictions))
    print(classification_report(y_test, predictions))
    print("-" * 60)

    
# Function to evaluate model using KFold Cross Validation
def evaluate_model_kfold(clf, X, y, n_splits=5, model_type="classification"):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        if model_type == "regression":
            predictions = np.round(predictions)
        
        accuracies.append(accuracy_score(y_test, predictions))
        
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    return mean_acc, std_acc


def evaluate_with_cv(clf, X, y, cv_strategy, scoring='accuracy'):
    scores = cross_val_score(clf, X, y, cv=cv_strategy, scoring=scoring)
    return scores.mean(), scores.std()

# # Improved Regularization for Naive Bayes (GaussianNB)
# nb_clf = GaussianNB(var_smoothing=1e-9)  # Smoothing to avoid zero variance issues

# # Improved Regularization for RandomForest
# rf_reg = RandomForestClassifier(
#     max_depth=10,  # Limit depth to prevent overfitting
#     min_samples_split=5,  # Minimum samples to split node
#     min_samples_leaf=2,  # Minimum samples in leaf node
#     n_estimators=100,  # Use 100 trees
#     random_state=42
# )

# dct_reg = DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)


# # Classifiers to try
# classifiers = [
#     DummyClassifier(strategy="most_frequent"),
#     DecisionTreeClassifier(),
#     dct_reg,
#     RandomForestClassifier(),
#     rf_reg,
#     KNeighborsClassifier(n_neighbors=5),
#     GaussianNB(),
#     nb_clf,
#     #LogisticRegression(max_iter=100),  # Increase from the default 100
#     LogisticRegression(max_iter=400),
#     Ridge(alpha=1.0),
#     Lasso(alpha=0.01)  # Tune alpha based on performance
# ]

# Updated Dummy Classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# Updated Decision Tree
dct_clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=2,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42
)

# Updated Random Forest
rf_clf = RandomForestClassifier(
    criterion="gini",
    max_depth=4,
    max_features="sqrt",
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

# Updated KNN
knn_clf = KNeighborsClassifier(
    metric="manhattan",
    n_neighbors=10
)

# Updated Naive Bayes
nb_clf = GaussianNB(var_smoothing=1e-9)  # Best parameters are default

# Updated Logistic Regression
lr_clf = LogisticRegression(
    C=1.0,
    solver="lbfgs",
    max_iter=400,
    random_state=42
)

# Classifiers list with updated configurations
classifiers = [
    dummy_clf,
    dct_clf,
    rf_clf,
    knn_clf,
    nb_clf,
    lr_clf
]

# Define the range of random states to test
random_states = range(1, 100)

# Store results for each classifier
results = {}

from sklearn.metrics import precision_score, recall_score, f1_score

# Function to evaluate across multiple random states
def evaluate_across_random_states(clf, X, y, random_states):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    
    for random_state in random_states:
        # Split the dataset with the current random_state
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random_state, stratify=y)
        
        if isinstance(clf, (Lasso, Ridge)):
            # Regression models
            clf.fit(X_train, y_train)
            predictions = np.round(clf.predict(X_test))
        else:
            # Classification models
            clf.fit(X_train, y_train)
            predictions = clf.predict(X_test)
        
        # Calculate metrics
        accuracies.append(accuracy_score(y_test, predictions))
        # Set zero_division=0 to handle undefined precision, recall, and f1 scores
        precisions.append(precision_score(y_test, predictions, average='weighted', zero_division=0))
        recalls.append(recall_score(y_test, predictions, average='weighted', zero_division=0))
        f1_scores.append(f1_score(y_test, predictions, average='weighted', zero_division=0))
    
    # Return mean and std for each metric
    return (np.mean(accuracies), np.std(accuracies),
            np.mean(precisions), np.std(precisions),
            np.mean(recalls), np.std(recalls),
            np.mean(f1_scores), np.std(f1_scores))

# Cross-validation results storage
cv_results = {}

# Stratified K-Fold (preserves class balance)
stratified_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Leave-One-Out CV
loo = LeaveOneOut()


# for clf in classifiers:
#     # Stratified K-Fold CV
#     mean_acc_skf, std_acc_skf = evaluate_with_cv(clf, X, y, stratified_kf)
    
#     # LOOCV (⚠️ will be slow if dataset is large)
#     mean_acc_loo, std_acc_loo = evaluate_with_cv(clf, X, y, loo)
    
#     cv_results[clf.__class__.__name__] = {
#         'stratified_kfold_accuracy': f"{mean_acc_skf:.4f} ± {std_acc_skf:.4f}",
#         'loocv_accuracy': f"{mean_acc_loo:.4f} ± {std_acc_loo:.4f}"
#     }

for clf in classifiers:
    # --- Cross-validation ---
    mean_acc_skf, std_acc_skf = evaluate_with_cv(clf, X, y, stratified_kf)
    mean_acc_loo, std_acc_loo = evaluate_with_cv(clf, X, y, loo)
    
    cv_results[clf.__class__.__name__] = {
        'stratified_kfold_accuracy': f"{mean_acc_skf:.4f} ± {std_acc_skf:.4f}",
        'loocv_accuracy': f"{mean_acc_loo:.4f} ± {std_acc_loo:.4f}"
    }

    # --- Random states ---
    (mean_acc, std_acc, mean_prec, std_prec, 
     mean_rec, std_rec, mean_f1, std_f1) = evaluate_across_random_states(clf, X, y, random_states)
    
    results[clf.__class__.__name__] = {
        'mean_accuracy': mean_acc, 'std_accuracy': std_acc,
        'mean_precision': mean_prec, 'std_precision': std_prec,
        'mean_recall': mean_rec, 'std_recall': std_rec,
        'mean_f1': mean_f1, 'std_f1': std_f1
    }


# Print the CV results
print("\n=== Cross-Validation Results ===")
for clf_name, metrics in cv_results.items():
    print(f"Classifier: {clf_name}")
    print(f"Stratified K-Fold Accuracy: {metrics['stratified_kfold_accuracy']}")
    print(f"LOOCV Accuracy: {metrics['loocv_accuracy']}")
    print("-" * 60)

# Print the results for each classifier
for clf_name, metrics in results.items():
    print(f"Classifier: {clf_name}")
    print(f"Average Accuracy: {metrics['mean_accuracy']:.4f} ± {metrics['std_accuracy']:.4f}")
    print(f"Average Precision: {metrics['mean_precision']:.4f} ± {metrics['std_precision']:.4f}")
    print(f"Average Recall: {metrics['mean_recall']:.4f} ± {metrics['std_recall']:.4f}")
    print(f"Average F1 Score: {metrics['mean_f1']:.4f} ± {metrics['std_f1']:.4f}")
    print("-" * 60)



['Gender', 'Age', 'Occupation', 'Marital status', 'HouseHoldIncome', 'IncomeGroup', 'Sleeping_duration_weekend', 'University_Class_duration_weekend', 'Study_duration_weekend', 'Exercise_duration_weekend', 'Work_duration_weekend', 'Housework_duration_weekend', 'Entertainment_duration_weekend', 'Personal_Care_duration_weekend', 'Eating_duration_weekend', 'Socializing_duration_weekend', 'Religion_or_Spiritual_Activities_duration_weekend', 'Shopping_duration_weekend', 'Cooking_duration_weekend', 'Family_Time_duration_weekend', 'Sports_duration_weekend', 'Reading_or_Writing_duration_weekend', 'Nightlife_duration_weekend', 'Travelling_duration_weekend', 'Volunteering_duration_weekend', 'Gardening_duration_weekend', 'Entertainment_merged_duration_weekend', 'Personal_care_merged_duration_weekend', 'Housework_merged_duration_weekend', 'Active_duration_weekend', 'Leisure_duration_weekend', 'Neutral_duration_weekend', 'Productivity_score_weekend']
['Gender', 'Age', 'Occupation', 'Marital status',

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

...

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis


=== Cross-Validation Results ===
Classifier: DummyClassifier
Stratified K-Fold Accuracy: 0.4888 ± 0.0072
LOOCV Accuracy: 0.4888 ± 0.4999
------------------------------------------------------------
Classifier: DecisionTreeClassifier
Stratified K-Fold Accuracy: 1.0000 ± 0.0000
LOOCV Accuracy: 1.0000 ± 0.0000
------------------------------------------------------------
Classifier: RandomForestClassifier
Stratified K-Fold Accuracy: 0.9911 ± 0.0178
LOOCV Accuracy: 0.9955 ± 0.0668
------------------------------------------------------------
Classifier: KNeighborsClassifier
Stratified K-Fold Accuracy: 0.5069 ± 0.0351
LOOCV Accuracy: 0.5067 ± 0.5000
------------------------------------------------------------
Classifier: GaussianNB
Stratified K-Fold Accuracy: 0.2822 ± 0.0369
LOOCV Accuracy: 0.2556 ± 0.4362
------------------------------------------------------------
Classifier: LogisticRegression
Stratified K-Fold Accuracy: 0.9238 ± 0.0104
LOOCV Accuracy: 0.9238 ± 0.2654
--------------------

C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Leeon\anaconda3\envs\498env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis